In [21]:
from pathlib import Path
from pprint import pprint

def list_daily_flow_files(base_dir: Path | None = None):
    """
    1. 取得 '每日各站進出站人數' 資料夾
    2. 取得其中所有 CSV 檔案的絕對路徑
    3. 排除 manifest.csv 與 schema.csv
    """
    if base_dir is None:
        base_dir = Path.cwd()  # 與 lesson19_2.ipynb 做法一致
    target_dir = base_dir / "每日各站進出站人數"
    if not target_dir.is_dir():
        raise FileNotFoundError(f"找不到資料夾: {target_dir}")

    excluded = {"manifest.csv", "schema.csv"}
    csv_paths = sorted(
        p.resolve()
        for p in target_dir.glob("*.csv")
        if p.name not in excluded
    )
    return target_dir.resolve(), csv_paths

folder_abs_path, file_abs_paths = list_daily_flow_files()

print("資料夾絕對路徑:")
print(folder_abs_path)
print("\n檔案絕對路徑 (已排除 manifest.csv, schema.csv):")
file_abs_paths = [str(p) for p in file_abs_paths]
print(file_abs_paths)

資料夾絕對路徑:
/Users/roberthsu2003/Documents/GitHub/__2025_06_25_aiAnalysis_1_3__/lesson19/每日各站進出站人數

檔案絕對路徑 (已排除 manifest.csv, schema.csv):
['/Users/roberthsu2003/Documents/GitHub/__2025_06_25_aiAnalysis_1_3__/lesson19/每日各站進出站人數/每日各站進出站人數20190423-20191231.csv', '/Users/roberthsu2003/Documents/GitHub/__2025_06_25_aiAnalysis_1_3__/lesson19/每日各站進出站人數/每日各站進出站人數2020.csv', '/Users/roberthsu2003/Documents/GitHub/__2025_06_25_aiAnalysis_1_3__/lesson19/每日各站進出站人數/每日各站進出站人數2021.csv', '/Users/roberthsu2003/Documents/GitHub/__2025_06_25_aiAnalysis_1_3__/lesson19/每日各站進出站人數/每日各站進出站人數2022.csv', '/Users/roberthsu2003/Documents/GitHub/__2025_06_25_aiAnalysis_1_3__/lesson19/每日各站進出站人數/每日各站進出站人數2023.csv']


In [19]:
import pandas as pd
current_dir = Path.cwd()
# 若 CSV 與此 notebook 同資料夾
csv_path = current_dir / "台鐵車站資訊.csv"
stations_df = pd.read_csv(csv_path)
#display(stations_df.head())
stations_df = stations_df.reindex(columns=["stationCode", "stationName"])

#欄位名稱更改為[車站代碼, 車站名稱]
stations_df.columns = ["車站代碼", "車站名稱"]
stations_df

,車站代碼,車站名稱
0,900,基隆
1,910,三坑
2,920,八堵
3,930,七堵
4,940,百福
...,...,...
238,7360,瑞芳
239,7361,海科館
240,7362,八斗子
241,7380,四腳亭


In [26]:
#建立一個function
#要concate下面迴圈的所有merged_df
def process_yearly_data(file_abs_paths, stations_df):
    merged_dfs = []
    for csv_path in file_abs_paths:
        year_df = pd.read_csv(csv_path)
        year_df.columns = ["日期", "車站代碼", "進站人數", "出站人數"]
        #display(year_df.head())
        #日期欄位目前是int64, 需要轉換為datetime格式
        year_df["日期"] = pd.to_datetime(year_df["日期"], format="%Y%m%d")
        merged_df = pd.merge(year_df, stations_df, on="車站代碼")
        merged_df = merged_df.reindex(columns=["日期","車站名稱","進站人數","出站人數"])
        merged_df.head()
        #將欄位:日期,變為index
        merged_df.set_index("日期", inplace=True)
        merged_dfs.append(merged_df)
    return pd.concat(merged_dfs)
result_df = process_yearly_data(file_abs_paths, stations_df)
result_df

,車站名稱,進站人數,出站人數
日期,,,
2019-04-23,基隆,8442,7743
2019-04-23,三坑,1394,1348
2019-04-23,八堵,2770,2423
2019-04-23,七堵,6113,6335
2019-04-23,百福,2680,2726
...,...,...,...
2023-12-31,瑞芳,7916,8252
2023-12-31,海科館,164,195
2023-12-31,八斗子,652,720


好的，這段程式碼定義了一個函式 `process_yearly_data`，其主要目的是讀取多個年度的車站進出站人數 CSV 檔案，將它們整理、合併，最後組合成一個完整的 DataFrame。

讓我們一步步來解析這個函式：

### 程式碼功能詳解

1.  **函式定義**
    ```python
    def process_yearly_data(file_abs_paths, stations_df):
    ```
    *   這裡定義了一個名為 `process_yearly_data` 的函式。
    *   它接受兩個參數：
        *   `file_abs_paths`: 一個包含多個 CSV 檔案絕對路徑的列表 (list)。
        *   `stations_df`: 一個包含車站代碼與車站名稱對應資訊的 DataFrame。

2.  **初始化與迴圈**
    ```python
    merged_dfs = []
    for csv_path in file_abs_paths:
        # ... 迴圈內的處理 ...
    ```
    *   `merged_dfs = []`: 建立一個空列表，用來存放處理完畢後、來自每一個 CSV 檔的 DataFrame。
    *   `for csv_path in file_abs_paths:`: 這是一個迴圈，它會逐一取出 `file_abs_paths` 列表中的每一個檔案路徑，然後執行迴圈內的程式碼。

3.  **資料讀取與清理 (迴圈內部)**
    ```python
    year_df = pd.read_csv(csv_path)
    year_df.columns = ["日期", "車站代碼", "進站人數", "出站人數"]
    year_df["日期"] = pd.to_datetime(year_df["日期"], format="%Y%m%d")
    ```
    *   `pd.read_csv(csv_path)`: 使用 pandas 讀取一個年度的 CSV 檔，存成 `year_df`。
    *   `year_df.columns = [...]`: 重新命名 `year_df` 的欄位，使其更具可讀性。
    *   `pd.to_datetime(...)`: 這是關鍵的一步。原始 "日期" 欄位是數字格式 (例如 `20220101`)，這裡將它轉換為 pandas 中專門處理時間序列的 `datetime` 格式，這樣後續才能進行基於時間的分析。`format="%Y%m%d"` 告訴 pandas 如何解析這個數字。

4.  **資料合併與重排 (迴圈內部)**
    ```python
    merged_df = pd.merge(year_df, stations_df, on="車站代碼")
    merged_df = merged_df.reindex(columns=["日期","車站名稱","進站人數","出站人數"])
    ```
    *   `pd.merge(...)`: 這就像是資料庫中的 `JOIN` 操作。它以 "車站代碼" 為共同的鍵，將 `year_df` (年度流量資料) 和 `stations_df` (車站基本資料) 合併。這樣做的結果是，我們可以為每一筆流量資料都加上對應的 "車站名稱"。
    *   `merged_df.reindex(...)`: 重新排列欄位的順序，並只保留指定的這四個欄位，讓 DataFrame 的結構更清晰。

5.  **設定索引與收集結果 (迴圈內部)**
    ```python
    merged_df.set_index("日期", inplace=True)
    merged_dfs.append(merged_df)
    ```
    *   `set_index("日期", inplace=True)`: 將 "日期" 欄位設定為 DataFrame 的索引 (Index)。這對於時間序列資料非常有用，可以讓我們更方便地按日期篩選或繪圖。`inplace=True` 表示直接在 `merged_df` 上修改，而不是回傳一個新的 DataFrame。
    *   `merged_dfs.append(merged_df)`: 將這個處理好的單一年度 DataFrame (`merged_df`) 加入到我們一開始建立的 `merged_dfs` 列表中。

6.  **最終合併與回傳**
    ```python
    return pd.concat(merged_dfs)
    ```
    *   迴圈結束後，`merged_dfs` 列表中會包含所有年份的 DataFrame。
    *   `pd.concat(merged_dfs)`: 這個指令會將列表中的所有 DataFrame 垂直堆疊起來，組合成一個包含所有年份資料的、單一且巨大的 DataFrame，並將其作為函式的結果回傳。

### 執行與結果



In [ ]:
result_df = process_yearly_data(file_abs_paths, stations_df)
result_df

*   這兩行程式碼呼叫了我們剛剛定義的函式，並將回傳的最終結果存放在 `result_df` 變數中。
*   最後一行 `result_df` 會在您的 Notebook 中顯示這個整理好的完整 DataFrame。

總結來說，這個儲存格的程式碼是一個非常實用的資料處理流程：**讀取多個來源檔案 -> 逐一清理與標準化 -> 合併補充資訊 -> 最終彙整成一個可用於分析的乾淨資料集**。